In [9]:
import pandas as pd
import numpy as np
from numpy import percentile
from scipy import stats
from scipy.stats import skew
from scipy.special import boxcox1p
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt


In [12]:
fuel_con = pd.read_csv('../data/ml_modules_data/co2_emissions.csv')

In [13]:
fuel_con.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [14]:
fuel_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Make                              7385 non-null   object 
 1   Model                             7385 non-null   object 
 2   Vehicle Class                     7385 non-null   object 
 3   Engine Size(L)                    7385 non-null   float64
 4   Cylinders                         7385 non-null   int64  
 5   Transmission                      7385 non-null   object 
 6   Fuel Type                         7385 non-null   object 
 7   Fuel Consumption City (L/100 km)  7385 non-null   float64
 8   Fuel Consumption Hwy (L/100 km)   7385 non-null   float64
 9   Fuel Consumption Comb (L/100 km)  7385 non-null   float64
 10  Fuel Consumption Comb (mpg)       7385 non-null   int64  
 11  CO2 Emissions(g/km)               7385 non-null   int64  
dtypes: flo

In [15]:
fuel_con.isnull().sum()

Make                                0
Model                               0
Vehicle Class                       0
Engine Size(L)                      0
Cylinders                           0
Transmission                        0
Fuel Type                           0
Fuel Consumption City (L/100 km)    0
Fuel Consumption Hwy (L/100 km)     0
Fuel Consumption Comb (L/100 km)    0
Fuel Consumption Comb (mpg)         0
CO2 Emissions(g/km)                 0
dtype: int64

In [16]:
fuel_con["Make"].nunique()

42

In [17]:
fuel_con["Make"].unique()

array(['ACURA', 'ALFA ROMEO', 'ASTON MARTIN', 'AUDI', 'BENTLEY', 'BMW',
       'BUICK', 'CADILLAC', 'CHEVROLET', 'CHRYSLER', 'DODGE', 'FIAT',
       'FORD', 'GMC', 'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP',
       'KIA', 'LAMBORGHINI', 'LAND ROVER', 'LEXUS', 'LINCOLN', 'MASERATI',
       'MAZDA', 'MERCEDES-BENZ', 'MINI', 'MITSUBISHI', 'NISSAN',
       'PORSCHE', 'RAM', 'ROLLS-ROYCE', 'SCION', 'SMART', 'SRT', 'SUBARU',
       'TOYOTA', 'VOLKSWAGEN', 'VOLVO', 'GENESIS', 'BUGATTI'],
      dtype=object)

In [18]:
fuel_con.Model.nunique()

2053

In [19]:
fuel_con["Vehicle Class"].nunique()

16

In [20]:
fuel_con["Vehicle Class"].unique()

array(['COMPACT', 'SUV - SMALL', 'MID-SIZE', 'TWO-SEATER', 'MINICOMPACT',
       'SUBCOMPACT', 'FULL-SIZE', 'STATION WAGON - SMALL',
       'SUV - STANDARD', 'VAN - CARGO', 'VAN - PASSENGER',
       'PICKUP TRUCK - STANDARD', 'MINIVAN', 'SPECIAL PURPOSE VEHICLE',
       'STATION WAGON - MID-SIZE', 'PICKUP TRUCK - SMALL'], dtype=object)

In [21]:
fuel_con["Transmission"].nunique()

27

In [22]:
fuel_con["Transmission"].unique()

array(['AS5', 'M6', 'AV7', 'AS6', 'AM6', 'A6', 'AM7', 'AV8', 'AS8', 'A7',
       'A8', 'M7', 'A4', 'M5', 'AV', 'A5', 'AS7', 'A9', 'AS9', 'AV6',
       'AS4', 'AM5', 'AM8', 'AM9', 'AS10', 'A10', 'AV10'], dtype=object)

In [23]:
# Mapping similar labels into a single label for Transmission column

fuel_con["Transmission"] = np.where(fuel_con["Transmission"].isin(["A4", "A5", "A6", "A7", "A8", "A9", "A10"]), "Automatic", fuel_con["Transmission"])
fuel_con["Transmission"] = np.where(fuel_con["Transmission"].isin(["AM5", "AM6", "AM7", "AM8", "AM9"]), "Automated Manual", fuel_con["Transmission"])
fuel_con["Transmission"] = np.where(fuel_con["Transmission"].isin(["AS4", "AS5", "AS6", "AS7", "AS8", "AS9", "AS10"]), "Automatic with Select Shift", fuel_con["Transmission"])
fuel_con["Transmission"] = np.where(fuel_con["Transmission"].isin(["AV", "AV6", "AV7", "AV8", "AV10"]), "Continuously Variable", fuel_con["Transmission"])
fuel_con["Transmission"] = np.where(fuel_con["Transmission"].isin(["M5", "M6", "M7"]), "Manual", fuel_con["Transmission"])

In [24]:
fuel_con["Transmission"].unique()

array(['Automatic with Select Shift', 'Manual', 'Continuously Variable',
       'Automated Manual', 'Automatic'], dtype=object)

In [25]:
fuel_con["Fuel Type"].nunique()

5

In [26]:
fuel_con["Fuel Type"].unique()

array(['Z', 'D', 'X', 'E', 'N'], dtype=object)

In [27]:
# Changing the labels into a meaningful label

fuel_con["Fuel Type"] = np.where(fuel_con["Fuel Type"]=="Z", "Premium Gasoline", fuel_con["Fuel Type"])
fuel_con["Fuel Type"] = np.where(fuel_con["Fuel Type"]=="X", "Regular Gasoline", fuel_con["Fuel Type"])
fuel_con["Fuel Type"] = np.where(fuel_con["Fuel Type"]=="D", "Diesel", fuel_con["Fuel Type"])
fuel_con["Fuel Type"] = np.where(fuel_con["Fuel Type"]=="E", "Ethanol(E85)", fuel_con["Fuel Type"])
fuel_con["Fuel Type"] = np.where(fuel_con["Fuel Type"]=="N", "Natural Gas", fuel_con["Fuel Type"])

In [28]:
fuel_con["Fuel Type"].unique()

array(['Premium Gasoline', 'Diesel', 'Regular Gasoline', 'Ethanol(E85)',
       'Natural Gas'], dtype=object)

In [29]:
fuel_con.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,Automatic with Select Shift,Premium Gasoline,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,Manual,Premium Gasoline,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,Continuously Variable,Premium Gasoline,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,Automatic with Select Shift,Premium Gasoline,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,Automatic with Select Shift,Premium Gasoline,12.1,8.7,10.6,27,244


1. There are total 42 types of car brand.

2. There are total 2053 unique car model. These neither can be converted into any dummy variable nor it can be used for analysis. So we can drop this column.

3. There are total 16 types of vehicle class basis on their gross vehicle weight rating (GVWR) and volume index. But there are no data available with exact GVWR or volume index value, so that we can categorise the similar vehicle into a same group.

4. The 27 type of transmission has been clubed into 5 different transmission without taking the number of clutches into account, as they doesnot affect CO2 emissions.

5. The 5 type of Fuel Types has been renamed so that it has some meaningful interpretation.
Frequency Distribution of different Categorical Variables


In [30]:
capacity_dict = {
    'COMPACT': 50,
    'SUV - SMALL': 60,
    'MID-SIZE': 70,
    'TWO-SEATER': 20,
    'MINICOMPACT': 40,
    'SUBCOMPACT': 50,
    'FULL-SIZE': 80,
    'STATION WAGON - SMALL': 50,
    'SUV - STANDARD': 70,
    'VAN - CARGO': 120,
    'VAN - PASSENGER': 60,  # Varies
    'PICKUP TRUCK - STANDARD': 100,
    'MINIVAN': 100,
    'SPECIAL PURPOSE VEHICLE': 40,  # Varies
    'STATION WAGON - MID-SIZE': 60,
    'PICKUP TRUCK - SMALL': 70
}

fuel_con['Capacity (cubic feet)'] = fuel_con['Vehicle Class'].map(capacity_dict)

transmission_labels = ['Automatic with Select Shift', 'Manual', 'Continuously Variable',
       'Automated Manual', 'Automatic'] 

fuel_labels = ['Z', 'D', 'X', 'E', 'N']

In [31]:
label_encoder = LabelEncoder()

fuel_con['Transmission'] = label_encoder.fit_transform(fuel_con['Transmission'])

transmission_label_mapping = {label: encoded_value for label, encoded_value in zip(transmission_labels, label_encoder.classes_)}
transmission_label_mapping

{'Automatic with Select Shift': 'Automated Manual',
 'Manual': 'Automatic',
 'Continuously Variable': 'Automatic with Select Shift',
 'Automated Manual': 'Continuously Variable',
 'Automatic': 'Manual'}

In [32]:
label_encoder = LabelEncoder()

fuel_con['Fuel Type'] = label_encoder.fit_transform(fuel_con['Fuel Type'])

fuel_type_label_mapping = {label: encoded_value for label, encoded_value in zip(fuel_labels, label_encoder.classes_)}
fuel_type_label_mapping

{'Z': 'Diesel',
 'D': 'Ethanol(E85)',
 'X': 'Natural Gas',
 'E': 'Premium Gasoline',
 'N': 'Regular Gasoline'}

In [33]:
fuel_con.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km),Capacity (cubic feet)
0,ACURA,ILX,COMPACT,2.0,4,2,3,9.9,6.7,8.5,33,196,50
1,ACURA,ILX,COMPACT,2.4,4,4,3,11.2,7.7,9.6,29,221,50
2,ACURA,ILX HYBRID,COMPACT,1.5,4,3,3,6.0,5.8,5.9,48,136,50
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,2,3,12.7,9.1,11.1,25,255,60
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,2,3,12.1,8.7,10.6,27,244,60


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X = fuel_con[['Capacity (cubic feet)', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type', 'Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)', 'Fuel Consumption Comb (mpg)']]
y = fuel_con['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1))  # Output layer for regression

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)

# Calculate the Mean Squared Error (MSE) as a performance metric
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared (R2) Score: {r2}")

# You can now use the model 'model' to make predictions on new data

Epoch 1/50
162/162 [==============================] - 3s 9ms/step - loss: 25727.3125 - val_loss: 3212.1189
Epoch 2/50
162/162 [==============================] - 1s 4ms/step - loss: 2510.4709 - val_loss: 1433.4868
Epoch 3/50
162/162 [==============================] - 0s 3ms/step - loss: 923.9745 - val_loss: 566.1329
Epoch 4/50
162/162 [==============================] - 1s 3ms/step - loss: 482.8799 - val_loss: 447.8276
Epoch 5/50
162/162 [==============================] - 0s 3ms/step - loss: 428.5174 - val_loss: 425.2454
Epoch 6/50
162/162 [==============================] - 0s 2ms/step - loss: 412.9197 - val_loss: 417.3594
Epoch 7/50
162/162 [==============================] - 0s 2ms/step - loss: 402.2003 - val_loss: 407.7342
Epoch 8/50
162/162 [==============================] - 0s 2ms/step - loss: 387.2192 - val_loss: 396.8478
Epoch 9/50
162/162 [==============================] - 1s 5ms/step - loss: 360.9865 - val_loss: 354.6940
Epoch 10/50
162/162 [==============================] - 0s 2

In [35]:
model.save('models/emission_model')

INFO:tensorflow:Assets written to: models/emission_model\assets


INFO:tensorflow:Assets written to: models/emission_model\assets


In [36]:
import tensorflow as tf
import numpy as np

# sample = np.array([['Capacity (cubic feet)', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type', 'Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)', 'Fuel Consumption Comb (mpg)']])
# example - get_emission_prediction(np.array([[50, 2.0, 4, 0, 2, 9.9, 7.4, 8.8, 32]]))
def get_emission_prediction(sample):
        loaded_model = tf.keras.models.load_model('models/emission_model')
        return loaded_model.predict(sample).tolist()[0][0]

In [37]:
get_emission_prediction(np.array([[50, 2.0, 4, 0, 2, 9.9, 7.4, 8.8, 32]]))

1/1 [==============================] - 0s 60ms/step


207.98968505859375